In [1]:
%matplotlib inline

# 神经网络  

可以使用torch.nn包来构建神经网络.  

你已知道autograd包,nn包依赖autograd包来定义模型并求导.一个nn.Module包含各个层和一个forward(input)方法,该方法返回output.  

例如,我们来看一下下面这个分类数字图像的网络.

![neural](../assets/neural.png)

他是一个简单的前馈神经网络,它接受一个输入,然后一层接着一层的输入,直到最后得到结果。

神经网络的典型训练过程如下:

- 定义神经网络模型,它有一些可学习的参数(或者权重);  
- 在数据集上迭代;  
* 通过神经网络处理输入;  
- 计算损失(输出结果和正确值的差距大小)  
- 将梯度反向传播回网络参数;  
- 更新网络的权重,主要使用如下简单的更新原则:weight = weight - learning_rate * gradient  

## 定义网络  

定义一个网络

In [11]:
import torch
import torch.nn as nn 
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 -- 输入图像通道数， 6 -- 输出通道数， 5x5 -- square convolution（平方卷积）
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 仿射运算  -- an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 5*5 from image dimension(图像维度)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        #如果大小为正方形，则可以使用单个数字指定
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
                                # 展开除批次维度以外的所有维度
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)

        


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


你只需定义forward函数,backward函数(计算梯度)在使用autograd时自动为你创建.你可以在forward函数中使用Tensor的任何操作。

net.parameters()返回模型需要学习的参数。

In [12]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 5, 5])


构造一个随机的32*32的输入，注意:这个网络(LeNet)期望的输入大小是32*32.如果使用MNIST数据集来训练这个网络,请把图片大小重新调整到32*32.

In [16]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
out


tensor([[-0.0395, -0.0515,  0.1344,  0.0207, -0.0588, -0.0194,  0.0373,  0.0187,
          0.0234, -0.0002]], grad_fn=<AddmmBackward>)

将所有参数的梯度缓存清零,然后进行随机梯度的的反向传播.

In [17]:
net.zero_grad()
out.backward(torch.randn(1, 10))

> 注意：  
torch.nn仅支持小批量。整个torch.nn 包仅支持小批量样本的输入，而不支持单个样本。  
例如，nn.Conv2d将采用 4D 张量 。nSamples x nChannels x Height x Width  
如果您只有一个样本，只需使用input.unsqueeze(0)添加一个伪批次维度即可。


在继续之前，让我们回顾一下您目前看到的所有类。

**回顾**  

- torch.Tensor - 一个多维数组，支持像backward(). 还持有张量的梯度w.r.t.
- nn.Module - 神经网络模块。封装参数的便捷方式，带有将它们移动到 GPU、导出、加载等的帮助程序。
- nn.Parameter - 一种张量，当作为属性分配给 Module.
- autograd.Function - 实现autograd 操作的向前和向后定义。每个Tensor操作至少创建一个Function节点，该节点连接到创建一个 Tensor并编码其历史的函数。

在这一节，我们介绍了：
- 定义神经网络
- 处理输入并向后传递
还剩：
- 计算损失
- 更新网络的权重



## 损失函数  

损失函数采用（输出，目标）对输入，并计算一个值，该值估计输出与目标的距离。

nn 包下有几个不同的 损失函数。一个简单的损失是：`nn.MSELoss`它计算输入和目标之间的均方误差。

In [18]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.9755, grad_fn=<MseLossBackward>)


现在,你反向跟踪loss,使用它的.grad_fn属性,你会看到向下面这样的一个计算图:  

```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> flatten -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
      
```

所以, 当你调用loss.backward(),整个图被区分为损失以及图中所有具有requires_grad = True的张量，并且其.grad 张量的梯度累积。

为了说明,我们反向跟踪几步:

In [21]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

## Backprop -- 反向传播   

为了反向传播误差,我们所需做的是调用loss.backward().你需要清除已存在的梯度,否则梯度将被累加到已存在的梯度。

现在,我们将调用loss.backward(),并查看conv1层的偏置项在反向传播前后的梯度。

In [24]:
net.zero_grad() #将所有参数的渐变缓冲区归零  

print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

现在，我们已经看到了如何使用损失函数。

稍后阅读:
神经网络包包含了各种用来构成深度神经网络构建块的模块和损失函数,一份完整的文档查看[这里](https://pytorch.org/docs/nn)

## 更新权重

实践中使用的最简单的更新规则是随机梯度下降 (SGD)：

```
weight = weight - learning_rate * gradient
```
我们可以使用简单的 Python 代码来实现：

In [25]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

但是，当您使用神经网络时，您希望使用各种不同的更新规则，例如 SGD、Nesterov-SGD、Adam、RMSProp 等。为了实现这一点，我们构建了一个小包：torch.optim它实现了所有这些方法。使用它非常简单：

In [26]:
import torch.optim as optim

# 创建你的优化器
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad() # zero the gradient buffers -- 梯度缓冲区归零
output = net(input)
loss = criterion(output, target)
loss.backward() 
optimizer.step() # Does the update

**注意**
> 观察如何使用optimizer.zero_grad()手动将梯度缓冲区设置为零。 这是因为梯度是反向传播部分中的说明那样是累积的。